In [1]:
import os
import numpy as np
import scipy
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import rtdl
from catboost.datasets import epsilon
import sklearn
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import zero
import sys
sys.path.append("../attacks/")
from utils import convert_relu_to_softplus
%matplotlib inline

In [2]:
# set seed:
zero.improve_reproducibility(123456)

123456

In [3]:
os.listdir("data/Learning to Rank Challenge/")

['set1.train.txt',
 'set1.test.txt',
 'README',
 'set2.test.txt',
 'set2.train.txt',
 'test_data1.csv',
 'WebscopeReadMe.txt',
 'set2.valid.txt',
 'set1.valid.txt',
 'ltrc_yahoo.tar.bz2']

In [5]:
with open("data/Learning to Rank Challenge/set2.train.txt", "r") as f:
    lines = f.readlines()
train2_dict = []
# we do not need query id (qid) for training
for l in lines:
    train2_dict.append({int(e.split(":")[0]):float(e.split(":")[1]) for e in l.strip().split(" ")[2:]})
    train2_dict[-1]["class"] = int(l.strip().split(" ")[0])
    
with open("data/Learning to Rank Challenge/set2.valid.txt", "r") as f:
    lines = f.readlines()
val2_dict = []
for l in lines:
    val2_dict.append({int(e.split(":")[0]):float(e.split(":")[1]) for e in l.strip().split(" ")[2:]})
    val2_dict[-1]["class"] = int(l.strip().split(" ")[0])
    
with open("data/Learning to Rank Challenge/set2.test.txt", "r") as f:
    lines = f.readlines()
test2_dict = []
for l in lines:
    test2_dict.append({int(e.split(":")[0]):float(e.split(":")[1]) for e in l.strip().split(" ")[2:]})
    test2_dict[-1]["class"] = int(l.strip().split(" ")[0])

In [6]:
train2 = pd.DataFrame(train2_dict)
val2 = pd.DataFrame(val2_dict)
test2 = pd.DataFrame(test2_dict)

In [4]:
with open("data/Learning to Rank Challenge/set1.train.txt", "r") as f:
    lines = f.readlines()
train1_dict = []
# we do not need query id (qid) for training
for l in lines:
    train1_dict.append({int(e.split(":")[0]):float(e.split(":")[1]) for e in l.strip().split(" ")[2:]})
    train1_dict[-1]["class"] = int(l.strip().split(" ")[0])
    
with open("data/Learning to Rank Challenge/set1.valid.txt", "r") as f:
    lines = f.readlines()
val1_dict = []
for l in lines:
    val1_dict.append({int(e.split(":")[0]):float(e.split(":")[1]) for e in l.strip().split(" ")[2:]})
    val1_dict[-1]["class"] = int(l.strip().split(" ")[0])
    
with open("data/Learning to Rank Challenge/set1.test.txt", "r") as f:
    lines = f.readlines()
test1_dict = []
for l in lines:
    test1_dict.append({int(e.split(":")[0]):float(e.split(":")[1]) for e in l.strip().split(" ")[2:]})
    test1_dict[-1]["class"] = int(l.strip().split(" ")[0])

In [5]:
train1 = pd.DataFrame(train1_dict)
val1 = pd.DataFrame(val1_dict)
test1 = pd.DataFrame(test1_dict)

In [6]:
train1_labels = train1["class"]
val1_labels = val1["class"]
test1_labels = test1["class"]

In [7]:
train1.drop("class", axis=1, inplace=True)
val1.drop("class", axis=1, inplace=True)
test1.drop("class", axis=1, inplace=True)

In [8]:
all_1 = pd.concat([train1, val1, test1], ignore_index = True, sort = False)

In [9]:
train1.fillna(0, inplace=True)
val1.fillna(0, inplace=True)
test1.fillna(0, inplace=True)

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [11]:
# data
task_type = "multiclass"

# X_tr = eps_train.iloc[:, 1:].astype('float32')
# y_tr = eps_train.iloc[:, 0].astype('float32' if task_type == 'regression' else 'int64')
# #
# X_te = eps_test.iloc[:, 1:].astype('float32')
# y_te = eps_test.iloc[:, 0].astype('float32' if task_type == 'regression' else 'int64')
if task_type != 'regression':
    train1_labels = LabelEncoder().fit_transform(train1_labels).astype('int64')
    val1_labels = LabelEncoder().fit_transform(val1_labels).astype('int64')
    test1_labels = LabelEncoder().fit_transform(test1_labels).astype('int64')
n_classes = int(max(train1_labels)) + 1 if task_type == 'multiclass' else None
print(n_classes)

X = {}
y = {}

X['test'] = test1
y['test'] = test1_labels

X["train"] = train1
y["train"] = train1_labels

X["val"] = val1
y["val"] = val1_labels


X = {
    k: torch.tensor(v.to_numpy(), device=device).float()
    for k, v in X.items()
}
y = {k: torch.tensor(v, device=device) for k, v in y.items()}


5


In [12]:
X["train"].dtype

torch.float32

In [19]:
d_out = n_classes or 1

model = rtdl.MLP.make_baseline(
    d_in=train1.shape[1],
    d_layers=[1024, 512, 256],
    dropout=0.45324571411572706,
    d_out=d_out,
)
lr = 0.003155649081268571
weight_decay = 4.5660291026998465e-06

# model = rtdl.ResNet.make_baseline(
#     d_in=train1.shape[1],
#     d_main=128,
#     d_hidden=701,
#     dropout_first=0.07437046655876002,
#     dropout_second=0.0,
#     n_blocks=9,
#     d_out=d_out,
# )
# lr = 1.1360909249836594e-05
# weight_decay = 1.0693403746836917e-06

# model = rtdl.FTTransformer.make_default(
#     n_num_features=X_all.shape[1],
#     cat_cardinalities=None,
#     last_layer_query_idx=[-1],  # it makes the model faster and does NOT affect its output
#     d_out=d_out,
# )

model.to(device)
optimizer = (
    model.make_default_optimizer()
    if isinstance(model, rtdl.FTTransformer)
    else torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
)
loss_fn = (
    F.binary_cross_entropy_with_logits
    if task_type == 'binclass'
    else F.cross_entropy
    if task_type == 'multiclass'
    else F.mse_loss
)

In [20]:
model

MLP(
  (blocks): Sequential(
    (0): Block(
      (linear): Linear(in_features=519, out_features=1024, bias=True)
      (activation): ReLU()
      (dropout): Dropout(p=0.45324571411572706, inplace=False)
    )
    (1): Block(
      (linear): Linear(in_features=1024, out_features=512, bias=True)
      (activation): ReLU()
      (dropout): Dropout(p=0.45324571411572706, inplace=False)
    )
    (2): Block(
      (linear): Linear(in_features=512, out_features=256, bias=True)
      (activation): ReLU()
      (dropout): Dropout(p=0.45324571411572706, inplace=False)
    )
  )
  (head): Linear(in_features=256, out_features=5, bias=True)
)

In [21]:
def apply_model(x_num, x_cat=None):
    if isinstance(model, rtdl.FTTransformer):
        return model(x_num, x_cat)
    elif isinstance(model, (rtdl.MLP, rtdl.ResNet)):
        assert x_cat is None
        return model(x_num)
    else:
        raise NotImplementedError(
            f'Looks like you are using a custom model: {type(model)}.'
            ' Then you have to implement this branch first.'
        )


@torch.no_grad()
def evaluate(part):
    model.eval()
    prediction = []
    for batch in zero.iter_batches(X[part], 1024):
        prediction.append(apply_model(batch))
    prediction = torch.cat(prediction).squeeze(1).cpu().numpy()
    target = y[part].cpu().numpy()

    if task_type == 'binclass':
        prediction = np.round(scipy.special.expit(prediction))
        score = sklearn.metrics.accuracy_score(target, prediction)
    elif task_type == 'multiclass':
        prediction = prediction.argmax(1)
        score = sklearn.metrics.accuracy_score(target, prediction)
    else:
        assert task_type == 'regression'
        score = sklearn.metrics.mean_squared_error(target, prediction) ** 0.5 * y_std
    return score


# Create a dataloader for batches of indices
# Docs: https://yura52.github.io/zero/reference/api/zero.data.IndexLoader.html
batch_size = 256
train_loader = zero.data.IndexLoader(len(X['train']), batch_size, device=device)

# Create a progress tracker for early stopping
# Docs: https://yura52.github.io/zero/reference/api/zero.ProgressTracker.html
progress = zero.ProgressTracker(patience=20)

print(f'Test score before training: {evaluate("test"):.4f}')

Test score before training: 0.0900


In [22]:
n_epochs = 1000
report_frequency = len(X['train']) // batch_size // 5
for epoch in range(1, n_epochs + 1):
    for iteration, batch_idx in enumerate(train_loader):
        model.train()
        optimizer.zero_grad()
        x_batch = X['train'][batch_idx]
        y_batch = y['train'][batch_idx]
        loss = loss_fn(apply_model(x_batch).squeeze(1), y_batch)
        loss.backward()
        optimizer.step()
        if iteration % report_frequency == 0:
            print(f'(epoch) {epoch} (batch) {iteration} (loss) {loss.item():.4f}')

    val_score = evaluate('val')
    test_score = evaluate('test')
    print(f'Epoch {epoch:03d} | Validation score: {val_score:.4f} | Test score: {test_score:.4f}', end='')
    progress.update((-1 if task_type == 'regression' else 1) * val_score)
    if progress.success:
        print(' <<< BEST VALIDATION EPOCH', end='')
        torch.save(model, "models/rtdl_mlp_yahoo.pth")
    print()
    if progress.fail:
        break

(epoch) 1 (batch) 0 (loss) 1.6359
(epoch) 1 (batch) 369 (loss) 1.2213
(epoch) 1 (batch) 738 (loss) 0.9436
(epoch) 1 (batch) 1107 (loss) 1.2442
(epoch) 1 (batch) 1476 (loss) 1.1252
(epoch) 1 (batch) 1845 (loss) 1.0862
Epoch 001 | Validation score: 0.3532 | Test score: 0.3568 <<< BEST VALIDATION EPOCH
(epoch) 2 (batch) 0 (loss) 1.1559
(epoch) 2 (batch) 369 (loss) 1.1526
(epoch) 2 (batch) 738 (loss) 0.9109
(epoch) 2 (batch) 1107 (loss) 1.2566
(epoch) 2 (batch) 1476 (loss) 1.1477
(epoch) 2 (batch) 1845 (loss) 1.0428
Epoch 002 | Validation score: 0.3532 | Test score: 0.3605
(epoch) 3 (batch) 0 (loss) 1.0987
(epoch) 3 (batch) 369 (loss) 1.1573
(epoch) 3 (batch) 738 (loss) 0.9325
(epoch) 3 (batch) 1107 (loss) 1.2197
(epoch) 3 (batch) 1476 (loss) 1.1224
(epoch) 3 (batch) 1845 (loss) 0.9996
Epoch 003 | Validation score: 0.3532 | Test score: 0.3570 <<< BEST VALIDATION EPOCH
(epoch) 4 (batch) 0 (loss) 1.1307
(epoch) 4 (batch) 369 (loss) 1.1347
(epoch) 4 (batch) 738 (loss) 0.9248
(epoch) 4 (batch)

In [24]:
model = torch.load("models/rtdl_mlp_yahoo.pth")
print(f'Test score after training: {evaluate("test"):.4f}')

Test score after training: 0.3570


In [21]:
model(X["test"][:10]).argmax(dim=1)

tensor([1, 1, 1, 1, 2, 1, 1, 1, 1, 1], device='cuda:0')

In [106]:
import catboost

In [107]:
cat_classifier = catboost.CatBoostClassifier()

In [109]:
cat_classifier.fit(X["train"].detach().cpu().numpy(), y["train"].detach().cpu().numpy())

Learning rate set to 0.10833
0:	learn: 1.5281398	total: 362ms	remaining: 6m 1s
1:	learn: 1.4670145	total: 665ms	remaining: 5m 31s
2:	learn: 1.4176007	total: 972ms	remaining: 5m 22s
3:	learn: 1.3777016	total: 1.32s	remaining: 5m 30s
4:	learn: 1.3440759	total: 1.62s	remaining: 5m 23s
5:	learn: 1.3154088	total: 1.98s	remaining: 5m 27s
6:	learn: 1.2920579	total: 2.28s	remaining: 5m 23s
7:	learn: 1.2707275	total: 2.64s	remaining: 5m 27s
8:	learn: 1.2531752	total: 2.98s	remaining: 5m 28s
9:	learn: 1.2377635	total: 3.32s	remaining: 5m 28s
10:	learn: 1.2245054	total: 3.69s	remaining: 5m 31s
11:	learn: 1.2124816	total: 4.03s	remaining: 5m 31s
12:	learn: 1.2023985	total: 4.35s	remaining: 5m 30s
13:	learn: 1.1934742	total: 4.72s	remaining: 5m 32s
14:	learn: 1.1856848	total: 5.07s	remaining: 5m 33s
15:	learn: 1.1785596	total: 5.39s	remaining: 5m 31s
16:	learn: 1.1723168	total: 5.68s	remaining: 5m 28s
17:	learn: 1.1662012	total: 6.04s	remaining: 5m 29s
18:	learn: 1.1609215	total: 6.34s	remaining: 5

157:	learn: 1.0642853	total: 47.2s	remaining: 4m 11s
158:	learn: 1.0640834	total: 47.6s	remaining: 4m 11s
159:	learn: 1.0638268	total: 47.9s	remaining: 4m 11s
160:	learn: 1.0635292	total: 48.2s	remaining: 4m 11s
161:	learn: 1.0633242	total: 48.5s	remaining: 4m 10s
162:	learn: 1.0631268	total: 48.7s	remaining: 4m 10s
163:	learn: 1.0628916	total: 49s	remaining: 4m 9s
164:	learn: 1.0626727	total: 49.2s	remaining: 4m 9s
165:	learn: 1.0624077	total: 49.5s	remaining: 4m 8s
166:	learn: 1.0621680	total: 49.8s	remaining: 4m 8s
167:	learn: 1.0618388	total: 50.1s	remaining: 4m 8s
168:	learn: 1.0616160	total: 50.3s	remaining: 4m 7s
169:	learn: 1.0613827	total: 50.6s	remaining: 4m 7s
170:	learn: 1.0612109	total: 50.9s	remaining: 4m 6s
171:	learn: 1.0609169	total: 51.2s	remaining: 4m 6s
172:	learn: 1.0606499	total: 51.5s	remaining: 4m 6s
173:	learn: 1.0603761	total: 51.8s	remaining: 4m 5s
174:	learn: 1.0601577	total: 52s	remaining: 4m 5s
175:	learn: 1.0598722	total: 52.4s	remaining: 4m 5s
176:	learn

312:	learn: 1.0379686	total: 1m 31s	remaining: 3m 20s
313:	learn: 1.0378085	total: 1m 31s	remaining: 3m 20s
314:	learn: 1.0376825	total: 1m 32s	remaining: 3m 20s
315:	learn: 1.0376166	total: 1m 32s	remaining: 3m 19s
316:	learn: 1.0375289	total: 1m 32s	remaining: 3m 19s
317:	learn: 1.0373302	total: 1m 32s	remaining: 3m 19s
318:	learn: 1.0371825	total: 1m 33s	remaining: 3m 19s
319:	learn: 1.0370813	total: 1m 33s	remaining: 3m 18s
320:	learn: 1.0369399	total: 1m 33s	remaining: 3m 18s
321:	learn: 1.0368172	total: 1m 34s	remaining: 3m 18s
322:	learn: 1.0366755	total: 1m 34s	remaining: 3m 17s
323:	learn: 1.0365605	total: 1m 34s	remaining: 3m 17s
324:	learn: 1.0363954	total: 1m 34s	remaining: 3m 17s
325:	learn: 1.0362798	total: 1m 35s	remaining: 3m 16s
326:	learn: 1.0361293	total: 1m 35s	remaining: 3m 16s
327:	learn: 1.0360190	total: 1m 35s	remaining: 3m 16s
328:	learn: 1.0358211	total: 1m 36s	remaining: 3m 16s
329:	learn: 1.0357169	total: 1m 36s	remaining: 3m 15s
330:	learn: 1.0356087	total:

466:	learn: 1.0204961	total: 2m 15s	remaining: 2m 34s
467:	learn: 1.0203579	total: 2m 15s	remaining: 2m 34s
468:	learn: 1.0202571	total: 2m 15s	remaining: 2m 33s
469:	learn: 1.0201721	total: 2m 16s	remaining: 2m 33s
470:	learn: 1.0200978	total: 2m 16s	remaining: 2m 33s
471:	learn: 1.0200236	total: 2m 16s	remaining: 2m 32s
472:	learn: 1.0199388	total: 2m 16s	remaining: 2m 32s
473:	learn: 1.0198412	total: 2m 17s	remaining: 2m 32s
474:	learn: 1.0197510	total: 2m 17s	remaining: 2m 32s
475:	learn: 1.0196621	total: 2m 17s	remaining: 2m 31s
476:	learn: 1.0195588	total: 2m 18s	remaining: 2m 31s
477:	learn: 1.0194826	total: 2m 18s	remaining: 2m 31s
478:	learn: 1.0193844	total: 2m 18s	remaining: 2m 30s
479:	learn: 1.0193025	total: 2m 18s	remaining: 2m 30s
480:	learn: 1.0192127	total: 2m 19s	remaining: 2m 30s
481:	learn: 1.0191097	total: 2m 19s	remaining: 2m 29s
482:	learn: 1.0190412	total: 2m 19s	remaining: 2m 29s
483:	learn: 1.0189545	total: 2m 20s	remaining: 2m 29s
484:	learn: 1.0188527	total:

619:	learn: 1.0079540	total: 2m 58s	remaining: 1m 49s
620:	learn: 1.0078412	total: 2m 58s	remaining: 1m 48s
621:	learn: 1.0077135	total: 2m 58s	remaining: 1m 48s
622:	learn: 1.0076157	total: 2m 59s	remaining: 1m 48s
623:	learn: 1.0075821	total: 2m 59s	remaining: 1m 48s
624:	learn: 1.0075260	total: 2m 59s	remaining: 1m 47s
625:	learn: 1.0074429	total: 3m	remaining: 1m 47s
626:	learn: 1.0073693	total: 3m	remaining: 1m 47s
627:	learn: 1.0073173	total: 3m	remaining: 1m 47s
628:	learn: 1.0072462	total: 3m	remaining: 1m 46s
629:	learn: 1.0071333	total: 3m 1s	remaining: 1m 46s
630:	learn: 1.0069987	total: 3m 1s	remaining: 1m 46s
631:	learn: 1.0069048	total: 3m 1s	remaining: 1m 45s
632:	learn: 1.0068483	total: 3m 2s	remaining: 1m 45s
633:	learn: 1.0067666	total: 3m 2s	remaining: 1m 45s
634:	learn: 1.0066950	total: 3m 2s	remaining: 1m 44s
635:	learn: 1.0066308	total: 3m 2s	remaining: 1m 44s
636:	learn: 1.0065483	total: 3m 3s	remaining: 1m 44s
637:	learn: 1.0064802	total: 3m 3s	remaining: 1m 44s

772:	learn: 0.9964079	total: 3m 41s	remaining: 1m 5s
773:	learn: 0.9963397	total: 3m 42s	remaining: 1m 4s
774:	learn: 0.9962858	total: 3m 42s	remaining: 1m 4s
775:	learn: 0.9961964	total: 3m 42s	remaining: 1m 4s
776:	learn: 0.9961294	total: 3m 43s	remaining: 1m 4s
777:	learn: 0.9960503	total: 3m 43s	remaining: 1m 3s
778:	learn: 0.9960023	total: 3m 43s	remaining: 1m 3s
779:	learn: 0.9959234	total: 3m 43s	remaining: 1m 3s
780:	learn: 0.9958833	total: 3m 44s	remaining: 1m 2s
781:	learn: 0.9958520	total: 3m 44s	remaining: 1m 2s
782:	learn: 0.9957865	total: 3m 44s	remaining: 1m 2s
783:	learn: 0.9957262	total: 3m 44s	remaining: 1m 1s
784:	learn: 0.9956539	total: 3m 45s	remaining: 1m 1s
785:	learn: 0.9956138	total: 3m 45s	remaining: 1m 1s
786:	learn: 0.9955143	total: 3m 45s	remaining: 1m 1s
787:	learn: 0.9954793	total: 3m 46s	remaining: 1m
788:	learn: 0.9954407	total: 3m 46s	remaining: 1m
789:	learn: 0.9953433	total: 3m 46s	remaining: 1m
790:	learn: 0.9952857	total: 3m 46s	remaining: 60s
791:

929:	learn: 0.9864580	total: 4m 25s	remaining: 20s
930:	learn: 0.9864035	total: 4m 25s	remaining: 19.7s
931:	learn: 0.9863375	total: 4m 26s	remaining: 19.4s
932:	learn: 0.9862656	total: 4m 26s	remaining: 19.1s
933:	learn: 0.9862171	total: 4m 26s	remaining: 18.8s
934:	learn: 0.9861696	total: 4m 26s	remaining: 18.6s
935:	learn: 0.9861212	total: 4m 27s	remaining: 18.3s
936:	learn: 0.9860589	total: 4m 27s	remaining: 18s
937:	learn: 0.9859976	total: 4m 27s	remaining: 17.7s
938:	learn: 0.9859145	total: 4m 28s	remaining: 17.4s
939:	learn: 0.9858649	total: 4m 28s	remaining: 17.1s
940:	learn: 0.9858147	total: 4m 28s	remaining: 16.8s
941:	learn: 0.9857612	total: 4m 28s	remaining: 16.6s
942:	learn: 0.9857156	total: 4m 29s	remaining: 16.3s
943:	learn: 0.9856730	total: 4m 29s	remaining: 16s
944:	learn: 0.9856008	total: 4m 29s	remaining: 15.7s
945:	learn: 0.9855456	total: 4m 30s	remaining: 15.4s
946:	learn: 0.9855067	total: 4m 30s	remaining: 15.1s
947:	learn: 0.9854226	total: 4m 30s	remaining: 14.8s

In [111]:
cat_classifier.score(X["test"].detach().cpu().numpy(), y["test"].detach().cpu().numpy())

0.3382409754919715

In [114]:
import sys
sys.path.append("../attacks/")
import mister_ed.utils.pytorch_utils as utils
from utils import load_image, torch_to_image, get_expl, convert_relu_to_softplus, plot_overview, UniGrad
import captum
from captum.attr import Saliency, InputXGradient, IntegratedGradients, DeepLift

In [115]:
model(X["test"][0:2])

tensor([[-1.6143],
        [ 1.6499]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [116]:
y["test"][0:2]

tensor([0, 1], device='cuda:0')

In [340]:
# sal = Saliency(model)
# expl = sal.attribute(X["test"][0:64])
# int_grad = IntegratedGradients(model)
# expl = int_grad.attribute(X["test"][0:64])
# inputxgrad = InputXGradient(model)
# expl = inputxgrad.attribute(X["test"][0:64])
deeplift = DeepLift(model)
expl = deeplift.attribute(X["test"][0:64])

In [344]:
expl = torch.abs(expl)
expl = expl / torch.sum(expl , dim=1, keepdim=True)

In [346]:
expl.size()

torch.Size([64, 2000])

In [342]:
topk = 10
topk_inds = torch.topk(expl, k=topk, dim=1)[1]

In [343]:
ints = []
for _ in range(30):
    e1, e2=np.random.randint(64, size=2)
#     print(e1, e2)
    ints.append(float(len(np.intersect1d(topk_inds[e1].cpu().detach().numpy(),
                            topk_inds[e2].cpu().detach().numpy())))/topk)
np.mean(ints)

0.2733333333333334

In [227]:
rand_input = (torch.zeros((1, 2000))).to(device)
expl_random = sal.attribute(rand_input)

In [228]:
print(float(len(np.intersect1d(topk_inds[e1].cpu().detach().numpy(),
                            torch.topk(expl_random, k=topk, dim=1)[1].cpu().detach().numpy())))/topk)

0.36


In [229]:
y["test"][:64]

tensor([0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0,
        1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1,
        0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0], device='cuda:0')

In [230]:
topk_inds.squeeze()

tensor([[1794,  690,  784,  ..., 1478, 1073, 1201],
        [1794, 1866,  484,  ..., 1296, 1029, 1678],
        [1794,  789, 1062,  ..., 1577,  723, 1976],
        ...,
        [1315, 1481,   62,  ...,  507,  824, 1800],
        [1697, 1794, 1062,  ...,   68,  308, 1759],
        [1794,  484, 1187,  ...,  972,  767,  784]], device='cuda:0')

In [51]:
def project_L0_box_torch(y, k, lb, ub):
    x = torch.clone(y)
    p1 = torch.sum(x**2, dim=-1)
    p2 = torch.minimum(torch.minimum(ub - x, x - lb), torch.zeros_like(x))
    p2 = torch.sum(p2**2, dim=-1)
    p3 = torch.sort(torch.reshape(p1-p2, (p2.size()[0],-1)))[0][:,-k]
    x = x*(torch.logical_and(lb <=x, x <= ub)) + lb*(lb > x) + ub*(x > ub)
    x = x * torch.unsqueeze((p1 - p2) >= p3.reshape([-1, 1, 1]), -1)

    return x

In [67]:
prj_x = project_L0_box_torch(X["test"][0:10].unsqueeze(dim=2).unsqueeze(dim=3), k=20, 
                     lb=torch.min(X["test"], dim=0)[0].unsqueeze(0).unsqueeze(dim=2).unsqueeze(dim=3), 
                     ub=torch.max(X["test"], dim=0)[0].unsqueeze(0).unsqueeze(dim=2).unsqueeze(dim=3))

In [69]:
prj_x = prj_x.squeeze(dim=3).squeeze(dim=2)

In [89]:
torch.max(X["test"], dim=0)[0]

tensor([0.1146, 0.0930, 0.0964,  ..., 0.0778, 0.1156, 0.0644], device='cuda:0')

In [88]:
F.sigmoid(model(X["test"][0:5]))

tensor([[0.1660],
        [0.8389],
        [0.9501],
        [0.0032],
        [0.6880]], device='cuda:0', grad_fn=<SigmoidBackward0>)

In [112]:
X["test"].max()

tensor(0.1446, device='cuda:0')

In [366]:
a = torch.tensor([[1,2,-4,-5],
                  [-0.7, 2, 1, -8],
                  [-1, 2, 0, 4]], device="cuda")
a

tensor([[ 1.0000,  2.0000, -4.0000, -5.0000],
        [-0.7000,  2.0000,  1.0000, -8.0000],
        [-1.0000,  2.0000,  0.0000,  4.0000]], device='cuda:0')

In [389]:
def topk_coord(array, used_inds, k=1):
    array_abs = torch.abs(array)
    if len(used_inds)>0:
        array_abs[np.array(used_inds).T.tolist()] = 0.0
    inds = torch.topk(array_abs.view((array.size()[0],-1)), k=k, dim=1)[1].detach().cpu().numpy()
    for k_ind in range(k):
        chosen_inds = [[i, j[k_ind]] for i,j in enumerate(inds)]
    ###
    new_inds = np.array([[i, j[0]] for i,j in enumerate(inds)])
    for k_ind in range(1,k):
        new_inds = np.concatenate((new_inds, np.array([[i, j[k_ind]] for i,j in enumerate(inds)])))
    ###
    new_inds = new_inds.T.tolist()
    ###
    new_array = torch.zeros_like(array)
    new_array[new_inds] = array[new_inds]
    ###
    return new_array, chosen_inds

In [401]:
a2, a_inds = topk_coord(a, [[1,3]])

In [406]:
b = a[np.array(a_inds).T.tolist()]
c = (b<-1) | (b>3)

In [408]:
np.array(a_inds)[c.detach().cpu().numpy()].tolist()

[[0, 3], [2, 3]]

In [411]:
np.array(a_inds).T[1]

array([3, 1, 3])

In [414]:
3.5>=3.5

True